https://python.langchain.com/docs/tutorials/agents/

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage
from enum import Enum
from langchain_google_genai import ChatGoogleGenerativeAI



OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY=os.getenv("TVLY_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
NVIDIA_API_KEY=os.getenv("NVIDIA_API_KEY")


In [ ]:
print(bool(OPENAI_API_KEY))
print(bool(GOOGLE_API_KEY))
print(bool(NVIDIA_API_KEY))

In [ ]:
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
#llm = init_chat_model("meta/llama3-70b-instruct", model_provider="nvidia")

Classify Text into Labels

Tagging means labeling a document with classes such as:

    Sentiment
    Language
    Style (formal, informal etc.)
    Covered topics
    Political tendency


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

In [ ]:
response.model_dump()

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

Finer control

Careful schema definition gives us more control over the model's output.

Specifically, we can define:

    Possible values for each property
    Description to make sure that the model understands the property
    Required properties to be returned

Let's redeclare our Pydantic model to control for each of the previously mentioned aspects using enums:

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)


In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

In [ ]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

In [ ]:
class SentimentEnum(str, Enum):
        happy = "happy"
        neutral = "neutral"
        sad = "sad"

class AggressivenessEnum(str, Enum):
        one = "1"
        two = "2"
        three = "3"
        four = "4"
        five = "5"

class LanguageEnum(str, Enum):
        spanish = "spanish"
        english = "english"
        french = "french"
        german = "german"
        italian = "italian"

class Classification(BaseModel):
        sentiment: SentimentEnum
        aggressiveness: AggressivenessEnum = Field(
            ..., description="describes how aggressive the statement is, the higher the number the more aggressive"
        )
        language: LanguageEnum

    # Structured LLM output
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash").with_structured_output(
        Classification
)

# Run
inp = "Quieres tomarte un cafe? Hay una nueva cafeteria que tiene muy buena pinta."
prompt = tagging_prompt.invoke({"input": inp})
result = llm.invoke(prompt)
print(result)
print(result.sentiment, result.aggressiveness, result.language)
